
<center>
<h4>Universidad Nacional de Córdoba - Facultad de Matemática, Astronomía, Física y Computación</h4>
<h3>Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones</h3>
 <h2>Mentoría: Clasificación de Tumoresferas </h2>
</center>



<a name="exploratory_data_analysis"></a>
#### **Práctico de Análisis Exploratorio y Visualización**



### Descripción de los datos: 

Se tratan de datos procesados en el [Grupo de Materia Condensada](https://ri.conicet.gov.ar/handle/11336/102) (FaMAF-UNC, IFEG-CONICET) en colaboración con el Laboratorio de Células Madre del [IBYME-CONICET](https://www.ibyme.org.ar/laboratorios/49/celulas-madre) a partir de imágenes tomadas usando un microscopio óptico invertido común.


Para extraer la información estructurada a partir de las imágenes se hizo uso del programa Fiji, que es un paquete de procesamiento de imágenes, distribución muy completa del programa [ImageJ](https://www.medicinatv.com/profesional/reportajes/una-plataforma-abierta-revoluciona-el-procesamiento-de-imagenes-biomedicas-4596/), que es ampliamente utilizado para procesar
imágenes médicas. 

De acuerdo a la documentación del programa,
es posible contar y medir objetos si conocemos la escala en la cual se han
llevado a cabo dichas imágenes, en este caso, a cuántos $\mu$m equivale un
píxel.
Para llevar a cabo el procedimiento detallado en la [documentación](https://imagej.nih.gov/ij/docs/guide/146-30.html#sub:Analyze-Particles...),
es necesario contar con una imagen umbralizada. 

Se realizó este procedimiento en las 91 imágenes disponibles y se etiquetaron en *esferoides* 'si' y 'no' de acuerdo al criterio de un [experto](https://www.conicet.gov.ar/new_scp/detalle.php?id=43977&datos_academicos=yes), obteniendo 1018 datos.



In [1]:
# Ejemplo de una de las imágenes umbralizadas:
from IPython.display import Image
Image(url="https://raw.githubusercontent.com/luciabarg/datos_tumoresferas/main/ejemplo_segmentacion.png", width = 800, height = 300)

Importamos las librerías necesarias:

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_context('talk') 
# por si quieren ver los distintos formatos:
# https://datavizpyr.com/seaborn-set_context-to-adjust-size-of-plot-labels-and-lines/
# https://seaborn.pydata.org/generated/seaborn.set_context.html

import warnings
warnings.filterwarnings("ignore")

## Datos


Cargamos los datos de los experimentos realizados desde el día cero al séptimo. 


Ya se encuentran las etiquetas y el valor de las poblaciones celulares estimadas en cada esferoide.

In [3]:
url = 'https://raw.githubusercontent.com/luciabarg/datos_tumoresferas/main/data/raw/fiji_datos_0al7mo_labels.csv' 
fiji_datos = pd.read_csv(url)
print('cantidad de datos:', len(fiji_datos))
fiji_datos.head()

cantidad de datos: 1018


,labels,Area,X,Y,XM,YM,Perim.,BX,BY,Width,...,FeretY,FeretAngle,MinFeret,AR,Round,Solidity,Esferoide,dia,Diameter,n_diam
0,Esferas_BT474_dia_0_well_1_100X_1_blob_1,324.4440,1129.2718,102.2051,1129.2718,102.2051,67.3603,1118.8859,92.3913,21.0598,...,157,23.4287,19.7011,1.0653,0.9387,0.9500,si,0,20.95640,1.491357
1,Esferas_BT474_dia_0_well_1_100X_1_blob_2,497.5115,1517.5528,126.1022,1517.5528,126.1022,82.8004,1504.7554,113.4511,25.8152,...,192,16.1443,25.1289,1.0475,0.9546,0.9569,si,0,26.00200,2.848733
2,Esferas_BT474_dia_0_well_1_100X_1_blob_3,282.9078,1314.2815,126.0988,1314.2815,126.0988,62.1586,1304.3478,116.8478,19.7011,...,189,15.4222,18.3424,1.0395,0.9620,0.9519,si,0,19.38970,1.181258
3,Esferas_BT474_dia_0_well_1_100X_1_blob_5,500.7421,1189.2841,212.6155,1189.2841,212.6155,84.5571,1175.9511,200.4076,26.4946,...,309,174.1440,25.1359,1.0278,0.9730,0.9493,si,0,25.88475,2.810370
4,Esferas_BT474_dia_0_well_1_100X_1_blob_6,492.8964,1043.0782,247.7667,1043.0782,247.7667,82.0728,1030.5707,235.7337,25.1359,...,353,143.4270,24.4565,1.0186,0.9817,0.9643,si,0,25.33985,2.636596


#### Descripción de las columnas:

   * **labels** (*'nombre_id'*): etiqueta que indica la estructura segmentada seleccionada. 
   * **Area** (*'area'*): columna que indica el área de selección en $\mu$m$^2$.
   * **Perim.** (*'perimetro'*): Es la longitud  del límite exterior de la selección. 
   * **Width** (*'ancho'*) y **Height** (*'altura'*): Es el ancho y la altura del rectángulo mas pequeño que encierra la estructura seleccionada.
   * **Circ.** (*'circulalidad'*): Indica la *circularidad*. Se calcula como 4π × [Area]/[Perimeter]$^2$, que con un valor de 1.0 indica un círculo perfecto. A medida que se acerca a 0.0, indica una forma cada vez más alargada. 
   * Columnas del diámetro de Feret: 
       * **Feret** (*feret*): Indica la distancia más larga entre dos puntos a lo largo del límite de selección. 
       * **MinFeret** (*min_feret*): Indica la distancia mínima entre dos puntos a lo largo del límite de selección.
       * **FeretAngle**: Indica el ángulo del diámetro de Feret.
       * **FeretX** y **FeretY**: son las coordenadas iniciales del diámetro de Feret. 
       * **AR** (*razon_aspecto*): Indica la  razón de aspecto de la elipse ajustada de la estructura seleccionada, es decir, [Eje mayor]/[Eje menor].
       * **Round** (*redondez*): Se calcula como  (4 × [Area] )/ (π × [Major axis]^2) o como la inversa de **AR**. Tiene un rango entre 0 y 1, con 1 indicando un círculo perfecto.
       * **Solidity**: Se calcula como [Area]/[Convex area]. Se calcula el área convexa como el área de una banda elástica envuelta firmemente alrededor de los puntos que definen la selección.
       * **Esferoide** (*esferoide*): etiqueta que indica si la estructura seleccionada califica como esferoide o no
       * **dia**: etiqueta que indica el día en el cual fue realizada la fotografía de origen. 
       * **Diameter** (*diametro*): Se calculó como 0.5*(*Feret*+*MinFeret*)
       * **n_diam** (*poblacion_celular*)


**Modelo para la estimación de la población celular *n_diam*:**

*Se puede hacer una estimación de la cantidad de células en una tumoresfera a partir del volumen $V_1$ de una célula de diámetro $D_1$;*

*Si tenemos una célula, entonces $V_1 = \frac{\pi}{6} D_1^3$ pero si el esferoide tiene $n$ células, entonces tendremos un volumen $V_n = n V_1$ para un diámetro $D_n$ de manera que $V_n =  \frac{\pi}{6} D_n^3$.*

*Si conocemos el diámetro $D_1$ de la célula a un tiempo inicial y el diámetro $D_n$ a un tiempo posterior, reemplazando $V_n$ y $V_1$ podemos estimar la población celular $n$ como: $n  = \frac{V_n}{V_1}= \frac{D_n^3}{D_0^3}.$*






Como utilizamos las imágenes ya segmentadas y recortadas, las columnas *X*, *Y*, *XM*, *YM*, *BX*, *BY*, las coordenadas del diámetro de Feret (*FeretX* y *FeretY*) y el ángulo (*FeretAngle*) **¿suman información al análisis?**
 

Para mas información, pueden explorar la [documentación](https://imagej.nih.gov/ij/docs/guide/146-30.html#sub:Analyze-Particles...).


Pueden ver las imágenes crudas y las individuales (cada fila es una estructura) en la siguiente [carpeta compartida](https://drive.google.com/drive/folders/1aws02t4Z0wSrdbHsQ-cgBkwX8PKLU71c?usp=sharing).

# Análisis de los datos:

*   1) a) Explorar cuántos datos son, ¿cuáles son las columnas? Diferenciar entre categóricas y numéricas.
    
       b) De acuerdo a la descripción de las columnas, definir cuáles son las más importantes y descartar las columnas que no tienen mucha validez para el problema.

*   2) Cambiar el nombre de las columnas en inglés a castellano (están indicadas en paréntesis en la descripción).
*   3) Tratemos de responder alguna de las siguientes preguntas:
    * a) ¿Se trata de una muestra o población?
    * b) ¿Es más probable que sea esferoide a que no lo sea?

    * c) ¿La forma (área, circularidad, diámetro, redondez) influye en la clasificación de *Esferoide*? 
    
    * d) ¿Tales variables son independientes entre sí? 
    * e) ¿La probabilidad de que la redondez ('Round') sea mayor a 0.9 en los clasificados como *Esferoides* = **si** es mayor que los clasificados como *Esferoides* = **no**?. 

* 4) También se puede explorar la muestra por día:
    * a) ¿Cuántos datos hay por día? Hacer gráfico de frecuencias.

    * b) Explorar con diagramas de cajas al menos una columna (ej. Diameter, n_diam, pero pueden animarse a más). 

    * c) Discriminar la(s) columna(s) elegida(s) en *Esferoide* **si** y **no** y ver sus distribuciones. ¿Son distintas?. Describirlas (asimetría, kurtosis, etc..)

    * d) ¿Hay outliers? Hacer con el total y luego discriminando por *Esferoide'* **si** y **no**. ¿Qué pueden implicar biológicamente? Si los eliminan, ¿cuál sería la justificación?

     Si identifican un par, traten de determinar el *'nombre_id'* y las buscamos en las imágenes en la [carpeta compartida](https://drive.google.com/drive/folders/1aws02t4Z0wSrdbHsQ-cgBkwX8PKLU71c?usp=sharing).

    * e) La media de los esferoides=*no* al día 3 es igual que al día 5? Realizar Test de Hipótesis. 
   
    * f) Determinen el intervalo de confianza diario de tal(es) columna(s) para las clasificadas como *'Esferoide'*=**'si'**. Y exploren como utilizarlos como barras de error en el gráfico. 
  
    El intervalo de confianza está dado por $$[LI;LS]$$ donde

    $$ LI = \overline{x}-t_{n-1,1-\alpha/2}\frac{s}{\sqrt n}$$
    $$ LS = \overline{x}+t_{n-1,1-\alpha/2}\frac{s}{\sqrt n}$$

   siendo $s$ la desviación de la distribución y $n$ el tamaño de la muestra.
   Pueden calcular el t-valor en la siguiente [página](https://goodcalculators.com/student-t-value-calculator/) y explorar la librería [scipy.stats.t](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.t.html) .



Interpretar lo que ven en los gráficos en términos generales. ¿Qué creen que sucedió a partir del día 6? 


* 5) ¿Qué otras preguntas se pueden o quisieran plantear? (las que puedan y quieran, las responden, sino planteenlas nomás y las charlamos, quizás se las puede responder con otras herramientas o simplemente son interesantes para tenerlas en cuenta y sería lindo tener otra clase de datos para responderlas.) 


Recordar que es importante como se comunican las respuestas, usar lo que van prendiendo a partir de cada punto y también guitarra ;).

Diseñar e implementar una comunicación a partir de algún ejercicio/s del entregable que les haya llamado la atención y que les parezca relevante en un archivo PDF o ppt.